In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os 
import pandas as pd
import numpy as np

import soepy

from dev_library import plot_basics_choices
from dev_library import plot_basics_wages
from dev_library import *
import pandas as pd
pd.set_option('display.max_rows', 500)
sys.path.insert(0, "/home/peisenha/structExpectations/development/estimations/basecamp")
os.chdir("/home/peisenha/structExpectations/development/estimations/basecamp")


In [13]:
params = pd.read_pickle("start.soepy.pkl")
params

value    upper    lower  fixed
category          name                                                       
benefits_base                             200.000000  1000.00  100.000   True
const_wage_eq     gamma_0s1                 1.734524     3.70    1.380  False
                  gamma_0s2                 2.273637     3.70    1.380  False
                  gamma_0s3                 2.134984     3.70    1.380  False
delta                                       0.980000     0.99    0.900  False
disutil_work      child_02_f                0.064536     3.00   -3.000  False
                  child_02_p                0.182351     3.00   -3.000  False
                  child_35_f               -0.129851     3.00   -3.000  False
                  child_35_p               -0.619096     3.00   -3.000  False
                  child_6orolder_f         -0.189406     3.00   -3.000  False
                  child_6orolder_p         -0.623263     3.00   -3.000  False
                  no_kids_f_educ_high       0.405765     3.00   -3.000  False
                  no_kids_f_educ_low        0.571940     3.00   -3.000  False
                  no_kids_f_educ_middle    -0.128656     3.00   -3.000  False
                  no_kids_p_educ_high      -0.285053     3.00   -3.000  False
                  no_kids_p_educ_low       -0.566743     3.00   -3.000  False
                  no_kids_p_educ_middle    -0.309465     3.00   -3.000  False
                  yes_kids_f_educ_high      1.097125     3.00   -3.000  False
                  yes_kids_f_educ_low       1.788128     3.00   -3.000  False
                  yes_kids_f_educ_middle    1.548187     3.00   -3.000  False
                  yes_kids_p_educ_high     -0.743666     3.00   -3.000  False
                  yes_kids_p_educ_low      -0.343479     3.00   -3.000  False
                  yes_kids_p_educ_middle   -0.295947     3.00   -3.000  False
exp_accm          g_s1                      0.102238     0.50    0.001  False
                  g_s2                      0.223013     0.50    0.001  False
                  g_s3                      0.261185     0.50    0.001  False
exp_accm_expected g_bar_s1                  0.570000     0.90    0.100   True
                  g_bar_s2                  0.535000     0.90    0.100   True
                  g_bar_s3                  0.625000     0.90    0.100   True
exp_deprec        delta_s1                  0.020000     0.20    0.001   True
                  delta_s2                  0.020000     0.20    0.001   True
                  delta_s3                  0.020000     0.20    0.001   True
exp_returns       gamma_1s1                 0.300524     0.50    0.001  False
                  gamma_1s2                 0.144266     0.50    0.001  False
                  gamma_1s3                 0.317441     0.50    0.001  False
hetrg_unobs       theta_f1                 -0.930271    -0.10   -3.000  False
                  theta_p1                 -0.685786    -0.10   -3.000  False
mu                                         -0.560000    -0.99   -0.010  False
sd_wage_shock     sigma_1                   0.354037     1.00    0.001  False
                  sigma_2                   0.392994     1.00    0.001  False
shares            share_1                   0.660619     0.90    0.100  False

In [10]:
os.chdir("/home/peisenha/structExpectations/development/estimations/estimation-current")

In [11]:
params = pd.read_pickle("start.soepy.pkl")
params

value    upper    lower  fixed
category          name                                                        
benefits_base                              200.000000  1000.00  100.000   True
const_wage_eq     gamma_0s1                  1.919595     3.70    1.380  False
                  gamma_0s2                  2.185583     3.70    1.380  False
                  gamma_0s3                  2.206118     3.70    1.380  False
delta                                        0.980000     0.99    0.900   True
exp_accm          g_s1                       0.068556     0.50    0.001  False
                  g_s2                       0.191744     0.50    0.001  False
                  g_s3                       0.276725     0.50    0.001  False
exp_deprec        delta_s1                   0.020000     0.20    0.001   True
                  delta_s2                   0.020000     0.20    0.001   True
                  delta_s3                   0.020000     0.20    0.001   True
exp_returns       gamma_1s1                  0.215364     0.50    0.001  False
                  gamma_1s2                  0.195151     0.50    0.001  False
                  gamma_1s3                  0.326145     0.50    0.001  False
fval              fval                    3856.281687      NaN      NaN   True
hetrg_unobs       theta_f1                  -0.994106    -0.10   -3.000  False
                  theta_p1                  -0.691135    -0.10   -3.000  False
mu                                          -0.560000    -0.99   -0.010   True
sd_wage_shock     sigma_1                    0.334074     1.00    0.001  False
                  sigma_2                    0.377055     1.00    0.001  False
shares            share_1                    0.660619     0.90    0.100   True
disutil_work      no_kids_f_educ_low         0.820924     3.00   -3.000  False
                  no_kids_f_educ_middle      0.380347     3.00   -3.000  False
                  no_kids_f_educ_high        0.444279     3.00   -3.000  False
                  yes_kids_f_educ_low        1.486966     3.00   -3.000  False
                  yes_kids_f_educ_middle     1.473071     3.00   -3.000  False
                  yes_kids_f_educ_high       1.921412     3.00   -3.000  False
                  child_02_f                 0.106891     3.00   -3.000  False
                  child_35_f                 0.019896     3.00   -3.000  False
                  child_610_f                0.005108     3.00   -3.000  False
                  no_kids_p_educ_low        -0.640720     3.00   -3.000  False
                  no_kids_p_educ_middle     -0.405925     3.00   -3.000  False
                  no_kids_p_educ_high       -0.390457     3.00   -3.000  False
                  yes_kids_p_educ_low       -0.196177     3.00   -3.000  False
                  yes_kids_p_educ_middle    -0.556250     3.00   -3.000  False
                  yes_kids_p_educ_high      -0.808112     3.00   -3.000  False
                  child_02_p                -0.207134     3.00   -3.000  False
                  child_35_p                -0.562927     3.00   -3.000  False
                  child_610_p               -0.612026     3.00   -3.000  False
exp_accm_expected g_bar_s1                   0.570000     0.90    0.100   True
                  g_bar_s2                   0.535000     0.90    0.100   True
                  g_bar_s3                   0.625000     0.90    0.100   True